In [11]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from difflib import SequenceMatcher
import os
# !pip install -q tqdm
from tqdm.notebook import tqdm

In [12]:
driver_options = Options()
driver_options.add_argument("accept-language=en-US")

s=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s,options=driver_options)
driver.maximize_window()

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\emman\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


In [18]:
INITIAL_WEBSITE = "https://oxtorrent.co"

In [20]:
driver.get(INITIAL_WEBSITE)

In [16]:
[cb.click() for cb in driver.find_elements(by=By.CSS_SELECTOR, value="dic.checkbox")]
[cb.click() for cb in driver.find_elements(by=By.CSS_SELECTOR, value="div.captcha")]

[]

In [46]:
header = driver.find_elements(by=By.CSS_SELECTOR, value="ul.arbo a")

categories = header[1:]
categories = {i.text: i.get_attribute('href') for i in categories}


for (category, link) in (pbar :=tqdm(categories.items())):
    # print(f"category {category} -- link: {link}")
    pbar.set_description("Processing category : %s" % category)
    next_page_link = link
    
    actual_page = 1
    is_first_page = True
    is_last_page_scraped = False
    list_of_tables = []
    
    while not is_last_page_scraped:
        driver.get(next_page_link)
        # first page processing
        if is_first_page:
            # print(f"first page processing")
            try:
                second_page_link = driver.find_element(by=By.CSS_SELECTOR, value="#contenu > div > div.main-wrap > ul > li:nth-child(2) > a").get_attribute('href')
                last_page_btn = driver.find_element(by=By.CSS_SELECTOR, value="#contenu > div > div.main-wrap > ul > li:last-child > a")
                last_page_link, n_pages = last_page_btn.get_attribute('href'), int(last_page_btn.text)
                match = SequenceMatcher(None, second_page_link, last_page_link).find_longest_match()
                base_link = second_page_link[match.a:match.a + match.size]
            except:
                n_pages = 1
            pbar_ = tqdm(total=n_pages, leave=False) # Progress bar of this category
            is_first_page = False # change the boolean to no more do the processing for the next pages
        # scrape content
        pbar_.set_description("Scraping page %s" % actual_page)
        # print("Scraping in processing")
        table = driver.find_element(by=By.CSS_SELECTOR, value="#contenu > div > div.main-wrap ")
        a_tags = table.find_elements(by=By.CSS_SELECTOR, value="table > tbody > tr > td > a")
        links = [a.get_attribute('href') for a in a_tags]
        columns=['torrent name', 'size', 'Seed', 'Leech' ]
        df = pd.read_html(table.get_attribute('innerHTML'), )[0]
        df.rename(columns={ df.columns.to_list()[i] : columns[i]for i in range(len(df.columns) )}, inplace=True)
        df["link"] = links
        list_of_tables.append(df)

        # checking
        if actual_page == n_pages :
            is_last_page_scraped = True
            # print("Last page scraped")
        else:
            # prepare to move to the next page
            actual_page += 1
            next_page_link = base_link+ f"{actual_page}"
            # print(f"Let's go to the page '{actual_page}' : {next_page_link}")
        pbar_.update(1)
        # print('')
        # break
    table_stack = pd.concat(list_of_tables, axis=0)
    n_items = table_stack.shape[0]
    table_stack.to_csv(os.path.join("data", f"{category}.csv"), index=False)
    del table_stack
    del list_of_tables
    pbar_.close()
    # break
    
    

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/574 [00:00<?, ?it/s]

  0%|          | 0/1036 [00:00<?, ?it/s]

  0%|          | 0/86 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [47]:
df.head()

,torrent name,size,Seed,Leech,link
0,One Shot FRENCH DVDRIP 2022,695.3 MB,124,316,https://www.oxtorrent.co/torrent/87182/one-sho...
1,Uncharted FRENCH DVDRIP 2022,692.2 MB,2083,954,https://www.oxtorrent.co/torrent/90367/unchart...
2,Spider-Man: No Way Home FRENCH DVDRIP 2021,1.4 GB,2003,364,https://www.oxtorrent.co/torrent/89219/spider-...
3,Uncharted TRUEFRENCH WEBRIP 2022,693.8 MB,1931,250,https://www.oxtorrent.co/torrent/90324/unchart...
4,The Batman TRUEFRENCH WEBRIP 2022,1.4 GB,1805,364,https://www.oxtorrent.co/torrent/90197/the-bat...


In [48]:
df.shape

(45, 5)